In [30]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

 # Data preparation

In [31]:
data_dir = 'dataset_Seminar5'
num_clients = 10
client_data = {}

# Load each client’s training data
for i in range(1, num_clients + 1):
    X_path = os.path.join(data_dir, f'client_datasets/client_{i}_features.csv')
    y_path = os.path.join(data_dir, f'client_datasets/client_{i}_labels.csv')
    
    X = pd.read_csv(X_path, header=None).values  # shape: (num_samples, 270)
    y = pd.read_csv(y_path, header=None).values.flatten()  # shape: (num_samples,)
    
    client_data[i] = {'X': X, 'y': y}

# Load test data
X_test = pd.read_csv(os.path.join(data_dir, 'test_features.csv'), header=None).values
y_test = pd.read_csv(os.path.join(data_dir, 'test_labels.csv'), header=None).values.flatten()

print(X.shape)  # shape: (num_samples, 270)
print(y.shape)  # shape: (num_samples,)
print(X_test.shape)  # shape: (num_samples, 270)
print(y_test.shape)  # shape: (num_samples,)

(64, 270)
(64,)
(500, 270)
(500,)


In [32]:
for client_id in client_data:
    client_data[client_id]['y'] -= 1  # Now ranges 0-11

y_test -= 1  # Also adjust test labels

Chosen architecture: CNN (Ressim())

In [33]:
class ResSim(nn.Module):
    """
    A simplified ResNet-style network with residual connections and sequential blocks.
    
    Adapted for CSI input (flattened to 3x30x3).

    Architecture:
        - Two residual blocks: Conv → ReLU → Conv + skip
        - Each followed by MaxPool
        - Fully connected classifier

    Args:
        num_classes (int): Number of output classes.
    """
    def __init__(self, num_classes=12): # 12 different pose classes
        super(ResSim, self).__init__()

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )
        self.shortcut1 = nn.Conv2d(3, 64, kernel_size=1)  # aligns input channels

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 1)
        self.fc = nn.Linear(64 * 7 * 4, num_classes)

    def forward(self, x):
        x = x.view(-1, 3, 30, 3)  # reshape input vector (270,) → (3, 30, 3)

        # First residual connection
        residual = self.shortcut1(x)
        x = self.block1(x)
        x = self.relu(x + residual)
        x = self.pool(x) # (64, 15, 3)

        # Second residual connection (no need for shortcut: same shape)
        residual = x
        x = self.block2(x)
        x = self.relu(x + residual)
        x = self.pool(x) # (64, 7, 3)

        x = x.view(x.size(0), -1) # flatten
        return self.fc(x)

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PoseClassifierCNN(nn.Module):
    def __init__(self, num_classes=12):
        super(PoseClassifierCNN, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),  # (3, 30, 3) → (16, 30, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=(2, 1)),  # (16, 15, 3)
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # (32, 15, 3)
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(2, 1))  # (32, 7, 3)
        )
        
        self.fc = nn.Sequential(
            nn.Flatten(),                    # (32 * 7 * 3)
            nn.Linear(32 * 7 * 3, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)     # output logits for 12 classes
        )

    def forward(self, x):
        # Reshape input from (batch_size, 270) → (batch_size, 3, 30, 3)
        x = x.view(-1, 3, 30, 3)  # match channel-first format
        x = self.cnn(x)
        x = self.fc(x)
        return x

In [35]:
class PoseClassifierFC(nn.Module):
    def __init__(self, num_classes=12):
        super(PoseClassifierFC, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(270, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x):
        return self.net(x)

In [36]:
# Federated Learning Parameters
NUM_CLIENTS = 10
CLIENTS_PER_ROUND = 5
FL_ROUNDS = 30

# Local training parameters
LOCAL_EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 0.001

In [ ]:
def eval_model(model, X, y):
    model.eval()
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    with torch.no_grad():
        logits = model(X_tensor)
        preds = torch.argmax(logits, dim=1)
        accuracy = ((preds == y_tensor).float().mean().item())*100
    return accuracy

In [45]:
def eval_loss(model, X, y, criterion):
    model.eval()
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    with torch.no_grad():
        logits = model(X_tensor)
        loss = criterion(logits, y_tensor)
    return loss.item()

def weighted_train_loss(model_class, global_state, selected_clients, client_data, criterion):
    train_losses = []
    weights = []
    for k in selected_clients:
        model = model_class(num_classes=12)
        model.load_state_dict(global_state)
        loss = eval_loss(model, client_data[k]['X'], client_data[k]['y'], criterion)
        train_losses.append(loss)
        weights.append(len(client_data[k]['X']))
    alpha = [w / sum(weights) for w in weights]
    weighted_loss = sum(a * l for a, l in zip(alpha, train_losses))
    return weighted_loss

In [48]:
import random
from copy import deepcopy

def initialize_global_model(model_class, num_classes=12):
    """Initialize the global model."""
    model = model_class(num_classes=num_classes)
    return model, deepcopy(model.state_dict())

def select_clients(client_ids, num_clients_per_round):
    """Randomly select a subset of clients for this round."""
    return random.sample(client_ids, num_clients_per_round)

def local_train(model_class, global_state, client_dataset, local_epochs, batch_size, lr):
    """Train a local model on a client's data."""
    model = model_class(num_classes=12)
    model.load_state_dict(global_state)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    X_local = torch.tensor(client_dataset['X'], dtype=torch.float32)
    y_local = torch.tensor(client_dataset['y'], dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(X_local, y_local)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for _ in range(local_epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
    return deepcopy(model.state_dict()), len(X_local)

def aggregate_models(local_states, local_sizes):
    """FedAvg aggregation of local model states."""
    total_samples = sum(local_sizes)
    new_global_state = deepcopy(local_states[0])
    for key in new_global_state:
        new_global_state[key] = sum(
            (local_states[i][key] * (local_sizes[i] / total_samples) for i in range(len(local_states)))
        )
    return new_global_state

def federated_learning(
    model_class, client_data, num_rounds, clients_per_round, local_epochs, batch_size, lr
):
    """Main FL orchestrator loop."""
    client_ids = list(client_data.keys())
    global_model, global_state = initialize_global_model(model_class)
    for round_idx in range(num_rounds):
        selected = select_clients(client_ids, clients_per_round)
        local_states, local_sizes = [], []
        for k in selected:
            state, size = local_train(
                model_class, global_state, client_data[k], local_epochs, batch_size, lr
            )
            local_states.append(state)
            local_sizes.append(size)
        global_state = aggregate_models(local_states, local_sizes)
        global_model.load_state_dict(global_state)
        print(f"Round {round_idx+1}/{num_rounds} complete.")

        if (round_idx + 1) % 5 == 0:
            # plot loss
            train_accuracy = eval_model(global_model, X_test, y_test)
            print(f"Round {round_idx+1}: Test Accuracy = {train_accuracy:.4f} %")
            loss = weighted_train_loss(
                model_class, global_state, selected, client_data, nn.CrossEntropyLoss()
            )
            print(f"Round {round_idx+1}: Weighted Train Loss = {loss:.4f}")
    return global_model

In [49]:
# Federated Learning Hyperparameters
NUM_CLIENTS = 10                # Number of clients participating in FL
CLIENTS_PER_ROUND = 8         # Number of clients selected per FL round
FL_ROUNDS = 50                 # Number of FL communication rounds
LOCAL_EPOCHS = 10                # Number of local epochs per client per round
BATCH_SIZE = 64                 # Batch size for local training
LEARNING_RATE = 0.001           # Learning rate for local optimizer

# Train the model using Federated Learning
global_model = federated_learning(
    model_class=PoseClassifierCNN,
    client_data=client_data,
    num_rounds=FL_ROUNDS,
    clients_per_round=CLIENTS_PER_ROUND,
    local_epochs=LOCAL_EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LEARNING_RATE
)

# Evaluate the trained global model on the test set
accuracy = eval_model(global_model, X_test, y_test)
print(f"Final Test Accuracy: {accuracy:.4f}")

Round 1/50 complete.
Round 2/50 complete.
Round 3/50 complete.
Round 4/50 complete.
Round 5/50 complete.
Round 5: Test Accuracy = 0.3780 %
Round 5: Weighted Train Loss = 1.1695
Round 6/50 complete.
Round 7/50 complete.
Round 8/50 complete.
Round 9/50 complete.
Round 10/50 complete.
Round 10: Test Accuracy = 0.4580 %
Round 10: Weighted Train Loss = 0.7650
Round 11/50 complete.
Round 12/50 complete.
Round 13/50 complete.
Round 14/50 complete.
Round 15/50 complete.
Round 15: Test Accuracy = 0.4680 %
Round 15: Weighted Train Loss = 0.7761
Round 16/50 complete.
Round 17/50 complete.
Round 18/50 complete.
Round 19/50 complete.
Round 20/50 complete.
Round 20: Test Accuracy = 0.4860 %
Round 20: Weighted Train Loss = 0.5871
Round 21/50 complete.
Round 22/50 complete.
Round 23/50 complete.
Round 24/50 complete.
Round 25/50 complete.
Round 25: Test Accuracy = 0.5280 %
Round 25: Weighted Train Loss = 0.4933
Round 26/50 complete.
Round 27/50 complete.
Round 28/50 complete.
Round 29/50 complete.
Rou

In [61]:
import random
from copy import deepcopy

# 1. Initialize a global ML model, ω(t = 0)
global_model = PoseClassifierCNN(num_classes=12)
global_model_state = deepcopy(global_model.state_dict())

# Helper: get number of samples for each client
client_num_samples = {k: len(client_data[k]['X']) for k in client_data}

# Federated Learning Loop
for round_idx in range(20):
    # 2. Select a subset of clients S ⊆ K
    selected_clients = random.sample(list(client_data.keys()), CLIENTS_PER_ROUND)
    
    local_states = []
    local_sizes = []
    
    # 3. Send the global model to the clients, retrain locally
    for k in selected_clients:
        local_model = PoseClassifierCNN(num_classes=12)
        local_model.load_state_dict(global_model_state)
        local_model.train()
        
        optimizer = torch.optim.Adam(local_model.parameters(), lr=LEARNING_RATE)
        criterion = nn.CrossEntropyLoss()
        
        X_local = torch.tensor(client_data[k]['X'], dtype=torch.float32)
        y_local = torch.tensor(client_data[k]['y'], dtype=torch.long)
        
        dataset = torch.utils.data.TensorDataset(X_local, y_local)
        loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        
        for epoch in range(LOCAL_EPOCHS):
            for xb, yb in loader:
                optimizer.zero_grad()
                logits = local_model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()
        
        # 4. Retrieve the individual models ωk(t)
        local_states.append(deepcopy(local_model.state_dict()))
        local_sizes.append(len(X_local))
    
    # 5. Aggregate the individual contributions (FedAvg)
    total_samples = sum(local_sizes)
    new_global_state = deepcopy(global_model_state)
    for key in new_global_state:
        new_global_state[key] = sum(
            (local_states[i][key] * (local_sizes[i] / total_samples) for i in range(len(local_states)))
        )
    global_model_state = new_global_state
    global_model.load_state_dict(global_model_state)
    
    print(f"Round {round_idx+1}/{FL_ROUNDS} complete.")

# The trained global_model now contains the aggregated weights

Round 1/50 complete.
Round 2/50 complete.
Round 3/50 complete.
Round 4/50 complete.
Round 5/50 complete.
Round 6/50 complete.
Round 7/50 complete.
Round 8/50 complete.
Round 9/50 complete.
Round 10/50 complete.
Round 11/50 complete.
Round 12/50 complete.
Round 13/50 complete.
Round 14/50 complete.
Round 15/50 complete.
Round 16/50 complete.
Round 17/50 complete.
Round 18/50 complete.
Round 19/50 complete.
Round 20/50 complete.


In [62]:
# Evaluate the global_model on the test set
global_model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

with torch.no_grad():
    logits = global_model(X_test_tensor)
    preds = torch.argmax(logits, dim=1)
    accuracy = (preds == y_test_tensor).float().mean().item()

print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5280
